In [63]:
#importing pandas, numpy and os
import pandas as pd
import numpy as np
import os

In [ ]:
#importing the latest ords_prods_merge pickle
path = "/Users/satoruteshima/Documents/CareerFoundry/Data Immersion 4/08-2023 Instacart Basket Analysis"

In [ ]:
ords_prods_merge = pd.read_pickle(os.path.join( path, '02 Data', '02 Prepared Data', 'orders_products_new_values_added.pkl'))

# TASKS

#### In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [ ]:
ords_prods_merge.groupby('department_id').agg({'order_number':['mean']})


#### Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

- The values are slightly different because it is aggregating the larger amount of data. 
- The top performing and bottom performing (by average) were the same 

#### Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.


In [ ]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [ ]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [ ]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [ ]:
#count values
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

#### The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [ ]:
#create a dataset only containing loyal customer 
loyal_customer = ords_prods_merge.loc[ords_prods_merge['loyalty_flag']== 'Loyal customer']

In [ ]:
#check shape to see if it matches with the value counts.
loyal_customer.shape

In [ ]:
#check the statistical data of prices of products purchased by loyal_customer 
loyal_customer.groupby('department_id').agg({'prices': ['mean', 'min', 'max']})

In [ ]:
#create a dataset NOTcontaining loyal customer 
other_customer = ords_prods_merge.loc[ords_prods_merge['loyalty_flag'] != 'loyal customer']


In [ ]:
#check the statistical data of prices of products purchased by loyal_customer 
other_customer.groupby('department_id').agg({'prices': ['mean', 'min', 'max']})

## Insights

- comparing the two datasets (loyal_customer and other_customer), we can see that there isn't a huge difference in the prices of products that they purchase. 

#### The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

- If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
- If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [ ]:
#create a new column max_price using transform function
ords_prods_merge['avg_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

create a new column 'spending_flag' using loc function

In [ ]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] < 10, 'spending_flag'] = 'Low spender'

In [ ]:
ords_prods_merge.loc[ords_prods_merge['avg_price'] >= 10, 'spending_flag'] = 'High spender'

In [60]:
#check value counts
ords_prods_merge['spending_flag'].value_counts(dropna = False)

Low spender     31770614
High spender      634245
Name: spending_flag, dtype: int64

#### In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:

- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”


In [67]:
ords_prods_merge['purchase_frequency'] = ords_prods_merge.groupby (['user_id'])['days_since_last_order'].transform(np.median)

In [89]:
ords_prods_merge.loc[ords_prods_merge['purchase_frequency']> 20, 'order_frequency_flag'] = 'Non-frequent customer'


In [81]:
ords_prods_merge.loc[(ords_prods_merge['purchase_frequency']> 10) & (ords_prods_merge['purchase_frequency'] <= 20), 'order_frequency_flag'] = 'Regular customer'

In [73]:
ords_prods_merge.loc[ords_prods_merge['purchase_frequency']<= 10, 'order_frequency_flag'] = 'Frequent customer'

In [90]:
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
Not enough data                 5
Name: order_frequency_flag, dtype: int64

In [91]:
ords_prods_merge.groupby(['order_frequency_flag']).user_id.nunique()

order_frequency_flag
Frequent customer        86596
Non-frequent customer    59619
Not enough data              1
Regular customer         59993
Name: user_id, dtype: int64

In [92]:
subset = ords_prods_merge[['user_id','days_since_last_order', 'order_frequency_flag']].head(10)
print(subset)

   user_id  days_since_last_order order_frequency_flag
0      138                    3.0    Frequent customer
1      138                   20.0    Frequent customer
2      709                    6.0    Frequent customer
3      764                    NaN    Frequent customer
4      764                    9.0    Frequent customer
5      777                   26.0     Regular customer
6      825                   30.0     Regular customer
7      910                   30.0    Frequent customer
8     1052                   19.0    Frequent customer
9     1052                   15.0    Frequent customer


In [95]:
#export
ords_prods_merge.to_pickle(os.path.join(path, '02 Data','02 Prepared Data', 'orders_products_more_flags_added.pkl'))